# Data preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/risklens/dataset


/content/drive/MyDrive/risklens/dataset


In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"johiyaharsh","key":"18543d1be6a071d8637591edec132867"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c home-credit-default-risk


 96% 663M/688M [00:04<00:00, 102MB/s]
100% 688M/688M [00:04<00:00, 156MB/s]


In [ ]:
!unzip home-credit-default-risk.zip


Archive:  home-credit-default-risk.zip
  inflating: HomeCredit_columns_description.csv  
  inflating: POS_CASH_balance.csv    
  inflating: application_test.csv    
  inflating: application_train.csv   
  inflating: bureau.csv              
  inflating: bureau_balance.csv      
  inflating: credit_card_balance.csv  
  inflating: installments_payments.csv  
  inflating: previous_application.csv  
  inflating: sample_submission.csv   


# **Data Overview**

In [ ]:
import os

BASE_PATH = "/content/drive/MyDrive/risklens/"
DATA_PATH = os.path.join(BASE_PATH, "dataset")

os.listdir(DATA_PATH)


['HomeCredit_columns_description.csv',
 'POS_CASH_balance.csv',
 'application_test.csv',
 'application_train.csv',
 'bureau.csv',
 'bureau_balance.csv',
 'credit_card_balance.csv',
 'installments_payments.csv',
 'previous_application.csv',
 'sample_submission.csv',
 'home-credit-default-risk.zip',
 'kaggle.json']

In [ ]:
import pandas as pd

app_train = pd.read_csv(
    os.path.join(DATA_PATH, "application_train.csv")
)

app_train.shape


(307511, 122)

In [ ]:
app_train['TARGET'].value_counts(normalize=True) * 100


,proportion
TARGET,
0,91.927118
1,8.072882


In [ ]:
app_train.info()
app_train.describe(include='all').T.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
SK_ID_CURR,307511.0,NaN,NaN,NaN,278180.518577,102790.175348,100002.0,189145.5,278202.0,367142.5,456255.0
TARGET,307511.0,NaN,NaN,NaN,0.080729,0.272419,0.0,0.0,0.0,0.0,1.0
NAME_CONTRACT_TYPE,307511,2,Cash loans,278232,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CODE_GENDER,307511,3,F,202448,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FLAG_OWN_CAR,307511,2,N,202924,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FLAG_OWN_REALTY,307511,2,Y,213312,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNT_CHILDREN,307511.0,NaN,NaN,NaN,0.417052,0.722121,0.0,0.0,0.0,1.0,19.0
AMT_INCOME_TOTAL,307511.0,NaN,NaN,NaN,168797.919297,237123.146279,25650.0,112500.0,147150.0,202500.0,117000000.0
AMT_CREDIT,307511.0,NaN,NaN,NaN,599025.999706,402490.776996,45000.0,270000.0,513531.0,808650.0,4050000.0
AMT_ANNUITY,307499.0,NaN,NaN,NaN,27108.573909,14493.737315,1615.5,16524.0,24903.0,34596.0,258025.5


In [ ]:
cols_desc = pd.read_csv(
    os.path.join(DATA_PATH, "HomeCredit_columns_description.csv"),
    encoding="latin1"
)

cols_desc.head()


,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


 # *Initial Observation* - application_train.csv ✅
1. Size: 307,511 rows × 122 features

2. Target: TARGET (1 = default, 0 = non-default)

3. Class imbalance:

Non-default ≈ 91.9%

Default ≈ 8.1% → accuracy not suitable; use AUC/KS/cost-based metrics

4. Feature types:

~106 numerical

16 categorical (loan type, gender, asset ownership, etc.)

5. Data quality:

Many features have high missing values; missingness likely informative

Financial variables are skewed with outliers

6. Leakage check:

No obvious post-outcome variables in application data; leakage will be controlled during feature engineering

